In [1]:
import numpy as np

In [2]:
# %run drive/MyDrive/Colab\ Notebooks/1_.ipynb

In [3]:
# %run drive/MyDrive/Colab\ Notebooks/2_binaryClassifier.ipynb

In [4]:
%run drive/MyDrive/Colab\ Notebooks/3_faults.ipynb

## ANN with a hidden layer

In [5]:
def alloc_param_pair(shape):
  weight = np.random.normal(RND_MEAN, RND_STD, shape)
  bias = np.zeros(shape[-1])
  return {'w':weight, 'b':bias}

def init_model_hidden_1():
  global pm_output, pm_hidden, input_cnt, output_cnt, hidden_cnt
  pm_hidden = alloc_param_pair((input_cnt, hidden_cnt))
  pm_output = alloc_param_pair((hidden_cnt, output_cnt))

In [6]:
def forward_neuralnet_hidden_1(x):
  global pm_output, pm_hidden

  hidden = relu(np.matmul(x, pm_hidden['w'])) + pm_hidden['b']
  output = np.matmul(hidden, pm_output['w']) + pm_output['b']
  return output, [x, hidden]

def relu(x):
  return np.maximum(x, 0)

def relu_derv(y):
  return np.sign(y)

In [7]:
prev_w = None

In [8]:
def backward_neuralnet_hidden_1(G_output, aux):
  global pm_output, pm_hidden
  global prev_w
  x, hidden = aux

  G_out_w = np.matmul(hidden.T, G_output)
  G_out_b = np.sum(G_output, axis=0)

  G_hidden = np.matmul(G_output, pm_output['w'].T)
  
  pm_output['w'] -= LEARNING_RATE * G_out_w
  pm_output['b'] -= LEARNING_RATE * G_out_b

  G_hidden = G_hidden * relu_derv(hidden)

  G_hidden_w = np.matmul(x.T, G_hidden)
  G_hidden_b = np.sum(G_hidden, axis=0)

  pm_hidden['w'] -= LEARNING_RATE * G_hidden_w
  pm_hidden['b'] -= LEARNING_RATE * G_hidden_b
  
  # print("#########################################")
  # if prev_w is not None:
  #   print((pm_hidden['w'] - prev_w).sum())
  # prev_w = pm_hidden['w']

## ANN with multi layers set by user

In [9]:
def init_model_hiddens():
  global pm_output, pm_hiddens, input_cnt, hidden_config, output_cnt
  pm_hiddens = []
  prev_cnt = input_cnt
  for hidden_cnt in hidden_config:
    pm_hiddens.append(alloc_param_pair((prev_cnt, hidden_cnt)))
    prev_cnt = hidden_cnt
  pm_output = alloc_param_pair((prev_cnt, output_cnt))

In [10]:
def forward_neuralnet_hiddens(x):
  global pm_output, pm_hiddens

  hidden = x
  hiddens = [x]
  for pm_hidden in pm_hiddens:
    hidden = relu(np.matmul(hidden, pm_hidden['w'])+ pm_hidden['b'])
    hiddens.append(hidden)
  output = np.matmul(hidden, pm_output['w']) + pm_output['b']
  return output, hiddens

In [11]:
def backward_neuralnet_hiddens(G_output, aux):
  global pm_output, pm_hiddens
  hiddens = aux

  G_output_w = np.matmul(hiddens[-1].T, G_output)
  G_output_b = np.sum(G_output, axis=0)

  G_hidden = np.matmul(G_output, pm_output['w'].T)

  pm_output['w'] -= LEARNING_RATE * G_output_w
  pm_output['b'] -= LEARNING_RATE * G_output_b

  for i in reversed(range(len(pm_hiddens))):
    G_hidden = G_hidden * relu_derv(hiddens[i+1])
    G_hidden_w = np.matmul(hiddens[i].T, G_hidden)
    G_hidden_b = np.sum(G_hidden, axis=0)

    G_hidden = np.matmul(G_hidden, pm_hiddens[i]['w'].T)

    pm_hiddens[i]['w'] -= LEARNING_RATE * G_hidden_w
    pm_hiddens[i]['b'] -= LEARNING_RATE * G_hidden_b

## SWITCH FUNCTION

In [12]:
def init_model():
  if hidden_config is not None:
    print('Multi layers network with {}-hidden layers is running.'.format(len(hidden_config)))
    init_model_hiddens()
  else:
    print("A layer network with one hidden layer is running.")
    init_model_hidden_1()
  
def forward_neuralnet(x):
  if hidden_config is not None:
    return forward_neuralnet_hiddens(x)
  else:
    print("check")
    return forward_neuralnet_hidden_1(x)

def backprop_neuralnet(G_output, hiddens):
  if hidden_config is not None:
    backward_neuralnet_hiddens(G_output, hiddens)
  else:
    backward_neuralnet_hidden_1(G_output, hiddens)

In [13]:
def set_hidden(info):
  global hidden_cnt, hidden_config
  if isinstance(info, int):
    hidden_cnt = info
    hidden_config = None
  else:
    hidden_config = info

In [19]:
set_hidden([10])
steel_exec(epoch_count=100, report=10)

Multi layers network with 1-hidden layers is running.
Epoch 10 | loss:1.935, accuracy=0.332
Epoch 20 | loss:1.924, accuracy=0.332
Epoch 30 | loss:1.912, accuracy=0.332
Epoch 40 | loss:1.901, accuracy=0.332
Epoch 50 | loss:1.891, accuracy=0.332
Epoch 60 | loss:1.883, accuracy=0.332
Epoch 70 | loss:1.872, accuracy=0.332
Epoch 80 | loss:1.863, accuracy=0.332
Epoch 90 | loss:1.856, accuracy=0.332
Epoch 100 | loss:1.848, accuracy=0.332
FINAL TEST result=0.332


In [25]:
LEARNING_RATE = 0.00001

In [26]:
set_hidden([12, 6, 4])
steel_exec(epoch_count=1000, report=10)

Multi layers network with 3-hidden layers is running.
Epoch 10 | loss:1.945, accuracy=0.363
Epoch 20 | loss:1.943, accuracy=0.363
Epoch 30 | loss:1.942, accuracy=0.363
Epoch 40 | loss:1.941, accuracy=0.363
Epoch 50 | loss:1.939, accuracy=0.363
Epoch 60 | loss:1.921, accuracy=0.363
Epoch 70 | loss:1.775, accuracy=0.363
Epoch 80 | loss:1.769, accuracy=0.363
Epoch 90 | loss:1.729, accuracy=0.343
Epoch 100 | loss:1.637, accuracy=0.202
Epoch 110 | loss:1.628, accuracy=0.233
Epoch 120 | loss:1.599, accuracy=0.212
Epoch 130 | loss:1.597, accuracy=0.215
Epoch 140 | loss:1.582, accuracy=0.248
Epoch 150 | loss:1.590, accuracy=0.327
Epoch 160 | loss:1.577, accuracy=0.289
Epoch 170 | loss:1.591, accuracy=0.299
Epoch 180 | loss:1.585, accuracy=0.355
Epoch 190 | loss:1.595, accuracy=0.297
Epoch 200 | loss:1.594, accuracy=0.332
Epoch 210 | loss:1.593, accuracy=0.358
Epoch 220 | loss:1.590, accuracy=0.435
Epoch 230 | loss:1.586, accuracy=0.322
Epoch 240 | loss:1.585, accuracy=0.325
Epoch 250 | loss:1.